In [ ]:
# Enhanced Eyewear Recommendation System with Face Shape Awareness
# Author: Frans Sebastian
import json
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import os
import random

# Initialize MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)

# Webcam Capture
def capture_face_from_camera():
    cap = cv2.VideoCapture(0)
    print("[INFO] Press 'c' to capture a photo or 'q' to quit.")
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
        cv2.imshow('Camera - Press c to capture', frame)
        key = cv2.waitKey(1)
        if key & 0xFF == ord('c'):
            img_path = 'captured_face.jpg'
            cv2.imwrite(img_path, frame)
            print(f"[INFO] Image saved to {img_path}")
            cap.release()
            cv2.destroyAllWindows()
            return img_path
        elif key & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    return None

# Extract landmarks
def extract_face_landmarks(image_path):
    image = cv2.imread(image_path)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb)
    h, w = image.shape[:2]
    if result.multi_face_landmarks:
        for face in result.multi_face_landmarks:
            landmarks = [(int(p.x * w), int(p.y * h)) for p in face.landmark]
            return landmarks
    return None

# Geometric features
def compute_geometric_features(landmarks):
    jaw_width = np.linalg.norm(np.array(landmarks[234]) - np.array(landmarks[454]))
    face_height = np.linalg.norm(np.array(landmarks[10]) - np.array(landmarks[152]))
    eye_distance = np.linalg.norm(np.array(landmarks[133]) - np.array(landmarks[362]))
    chin_width = np.linalg.norm(np.array(landmarks[58]) - np.array(landmarks[288]))
    ratio_jaw_to_height = jaw_width / face_height
    ratio_eye_to_height = eye_distance / face_height
    return {
        "jaw_width": jaw_width,
        "face_height": face_height,
        "eye_distance": eye_distance,
        "chin_width": chin_width,
        "ratio_jaw_to_height": ratio_jaw_to_height,
        "ratio_eye_to_height": ratio_eye_to_height
    }

# Classify face shape from ratios
def classify_face_shape(features):
    r1 = features["ratio_jaw_to_height"]
    r2 = features["ratio_eye_to_height"]
    if r1 > 0.85 and r2 < 0.3:
        return "round"
    elif r1 < 0.75 and r2 > 0.32:
        return "oval"
    elif r1 > 0.90:
        return "square"
    elif r1 < 0.70:
        return "heart"
    else:
        return "diamond"

# Face shape to frame shape mapping
face_shape_to_frames = {
    "round": ["rectangular", "cat-eye"],
    "oval": ["aviator", "round"],
    "square": ["round", "oval"],
    "heart": ["aviator", "cat-eye"],
    "diamond": ["oval", "rectangular"]
}

# Synthetic product data
styles = ["classic", "trendy", "minimal", "sporty", "fashion", "luxury", "bold"]
materials = ["plastic", "metal", "titanium", "acetate"]
brands = ["RayBan", "Gucci", "Local", "Oakley", "Prada", "Nike", "Zara", "Dior", "Tom Ford"]
colors = ["black", "gold", "transparent", "brown", "blue"]
frame_shapes = ["round", "rectangular", "cat-eye", "aviator"]
activities = ["office", "sport", "reading", "driving", "fashion"]
frame_sizes = ["S", "M", "L"]

products = []
for i in range(500):
    p = {
        "product_id": i + 1,
        "brand": random.choice(brands),
        "style": random.choice(styles),
        "material": random.choice(materials),
        "price": random.randint(700_000, 1_500_000),
        "weight_g": random.randint(20, 30),
        "lens_antirad": random.choice([True, False]),
        "lens_blue": random.choice([True, False]),
        "lens_photochromic": random.choice([True, False]),
        "frame_shape": random.choice(frame_shapes),
        "color": random.choice(colors),
        "popularity": random.randint(60, 95),
        "social_mentions": random.randint(100, 600),
        "rating": round(random.uniform(3.5, 5.0), 2),
        "stock": random.choice(["in", "out"]),
        "delivery_speed": random.choice([1, 2, 3, 5]),
        "durability": random.randint(70, 100),
        "warranty_years": random.choice([0, 1, 2]),
        "frame_size": random.choice(frame_sizes),
        "eco_friendly": random.choice([True, False]),
        "discount_percent": random.choice([0, 10, 15, 20]),
        "virtual_tryon": random.choice([True, False]),
        "activity": random.choice(activities)
    }
    products.append(p)
products = pd.DataFrame(products)

# Composite trend score
def compute_trend_score(df):
    features = df[["popularity", "social_mentions", "rating", "durability"]]
    scaler = StandardScaler()
    normed = scaler.fit_transform(features)
    df["composite_score"] = PCA(n_components=1).fit_transform(normed)
    return df

products = compute_trend_score(products)

# Recommendation engine
def recommend_products(df, preferences, face_shape, top_n=5):
    valid_shapes = face_shape_to_frames.get(face_shape, frame_shapes)
    filtered = df[
        (df.frame_shape.isin(valid_shapes)) &
        (df.style == preferences["style"]) &
        (df.material == preferences["material"]) &
        (df.price <= preferences["budget"]) &
        (df.weight_g <= preferences["max_weight"]) &
        (df.lens_antirad == preferences["lens_antirad"]) &
        (df.lens_blue == preferences["lens_blue"]) &
        (df.lens_photochromic == preferences["lens_photochromic"]) &
        (df.color == preferences["color"]) &
        (df.brand == preferences["brand"]) &
        (df.activity == preferences["activity"]) &
        (df.frame_size == preferences["frame_size"]) &
        (df.virtual_tryon == preferences["virtual_tryon"]) &
        (df.eco_friendly == preferences["eco_friendly"])
    ]
    if filtered.empty:
        print("[WARNING] No exact match found. Try relaxing filters.")
        return df[df.frame_shape.isin(valid_shapes)].sort_values("composite_score", ascending=False).head(top_n)
    return filtered.sort_values("composite_score", ascending=False).head(top_n)

# Main execution
if __name__ == "__main__":
    image_path = "captured_face.jpg" if os.path.exists("captured_face.jpg") else capture_face_from_camera()
    if not image_path:
        print("[INFO] No image captured.")
        exit()

    landmarks = extract_face_landmarks(image_path)
    if not landmarks:
        print("[ERROR] No face detected.")
        exit()

    features = compute_geometric_features(landmarks)
    face_shape = classify_face_shape(features)

    print("\n[RESULT] Facial Features:")
    for k, v in features.items():
        print(f"{k}: {v:.2f}")
    print(f"Detected face shape: {face_shape}")

    user_samples = [
        {
            "style": random.choice(styles),
            "material": random.choice(materials),
            "budget": random.randint(900_000, 1_500_000),
            "max_weight": random.randint(22, 27),
            "lens_antirad": random.choice([True, False]),
            "lens_blue": random.choice([True, False]),
            "lens_photochromic": random.choice([True, False]),
            "frame_shape": random.choice(frame_shapes),  # Ignored in recommendation, auto-filtered by face shape
            "color": random.choice(colors),
            "brand": random.choice(brands),
            "activity": random.choice(activities),
            "frame_size": random.choice(frame_sizes),
            "virtual_tryon": random.choice([True, False]),
            "eco_friendly": random.choice([True, False])
        }
        for _ in range(10)
    ]

    all_results = []

    for idx, prefs in enumerate(user_samples):
        print(f"\n[USER {idx+1} PREFERENCES]:")
        for k, v in prefs.items():
            print(f"{k}: {v}")

        recs = recommend_products(products, prefs, face_shape)
        print("\n[RECOMMENDED PRODUCTS]:")
        if recs.empty:
            print("No products matched all preferences.\n")
        else:
            print(recs.to_string(index=False))
        print("="*80)

        all_results.append({
            "user_id": idx+1,
            "face_shape": face_shape,
            "preferences": prefs,
            "recommendations": recs.to_dict(orient="records")
        })

    with open("eyewear_recommendation_results.json", "w") as f:
        json.dump(all_results, f, indent=4)



[RESULT] Facial Features:
jaw_width: 491.10
face_height: 618.03
eye_distance: 128.02
chin_width: 442.11
ratio_jaw_to_height: 0.79
ratio_eye_to_height: 0.21

[USER 1 PREFERENCES]:
style: luxury
material: metal
budget: 1196147
max_weight: 25
lens_antirad: True
lens_blue: False
lens_photochromic: True
frame_shape: aviator
color: gold
brand: Oakley
activity: fashion
frame_size: L
virtual_tryon: True
eco_friendly: False
[WARNING] No exact match found. Try relaxing filters.

[RECOMMENDED PRODUCTS]:
 product_id    brand   style material   price  weight_g  lens_antirad  lens_blue  lens_photochromic frame_shape       color  popularity  social_mentions  rating stock  delivery_speed  durability  warranty_years frame_size  eco_friendly  discount_percent  virtual_tryon activity  composite_score
        173 Tom Ford  luxury titanium 1265784        29         False      False              False     aviator        gold          62              583    3.53    in               5          93            

I0000 00:00:1750505867.678907  374787 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
W0000 00:00:1750505867.686133  382166 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750505867.693055  382166 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
